### Load model
- Catboost
loaded_cat = CatBoostRegressor()
loaded_cat.load_model("catboost_model.cbm")

- XGBoost
loaded_xgbr = XGBRegressor()
loaded_xgbr.load_model("xgboost_model.json")

- LightGBM
import lightgbm as lgb
loaded_lgbmr = lgb.Booster(model_file="lightgbm_model.txt")



In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import regex as re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
#import pygwalker as pyg

train = pd.read_csv("Data/Data.csv")
train.head()

,brand,price,old,new,cpu,cpu_brand,ram_capacity,ram_brand,hard_drive_type,hard_drive_capacity,card,card_brand,screen_size,screen_type
0,Asus,26990000.0,0,1,Intel Core Ultra 5 125H,Intel,16.0,DDR5,SSD,512GB,Intel Arc Graphics,Intel,14.0,OLED
1,Lenovo,16390000.0,0,1,AMD Ryzen 77730U,AMD,16.0,DDR4,SSD,512GB,AMD Radeon Graphics,AMD,15.6,HD
2,Asus,28990000.0,0,1,Intel Core Ultra 5 125H,Intel,16.0,DDR5,SSD,512GB,Intel Arc Graphics,Intel,14.0,OLED
3,LG,36290000.0,0,1,Intel Core Ultra 5 125H,Intel,16.0,DDR5,SSD,512GB,Intel Arc Graphics,Intel,14.0,IPS
4,Dell,18990000.0,0,1,Intel Core 5 processor 120U,Intel,8.0,DDR5,SSD,512GB,Intel Graphics,Intel,14.0,HD


In [31]:
train['brand'] = train['brand'].replace('Apple', 'Macbook')
train['brand'] = train['brand'].replace('MacBook', 'Macbook')
train['brand'] = train['brand'].replace('Thinkpad', 'Lenovo')
train['brand'] = train['brand'].replace('Legion', 'Lenovo')
train['brand'] = train['brand'].replace('Ideapad', 'Lenovo')
train['brand'] = train['brand'].replace('ThinkPad', 'Lenovo')
train['brand'] = train['brand'].replace('LENOVO', 'Lenovo')
train['brand'] = train['brand'].replace('ACER', 'Acer')
train['brand'] = train['brand'].replace('ASUS', 'Asus')


In [32]:
unique_brands = train['brand'].unique()
print(unique_brands)

['Asus' 'Lenovo' 'LG' 'Dell' 'HP' 'Acer' 'MSI' 'Macbook' 'Microsoft'
 'Gaming' nan]


In [33]:
brand_counts = train['brand'].value_counts()
print(brand_counts)

brand
Lenovo       696
Asus         356
HP           336
Dell         326
Acer         230
MSI          168
Macbook       38
LG            35
Gaming        17
Microsoft     10
Name: count, dtype: int64


In [34]:
train.columns

Index(['brand', 'price', 'old', 'new', 'cpu', 'cpu_brand', 'ram_capacity',
       'ram_brand', 'hard_drive_type', 'hard_drive_capacity', 'card',
       'card_brand', 'screen_size', 'screen_type'],
      dtype='object')

In [35]:
cat_cols = ['brand', 'cpu', 'cpu_brand', 'ram_capacity',
       'ram_brand', 'hard_drive_type', 'hard_drive_capacity', 'card',
       'card_brand', 'screen_size', 'screen_type']

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                2212 non-null   object 
 1   price                2141 non-null   float64
 2   old                  2215 non-null   int64  
 3   new                  2215 non-null   int64  
 4   cpu                  2206 non-null   object 
 5   cpu_brand            2213 non-null   object 
 6   ram_capacity         2209 non-null   float64
 7   ram_brand            2090 non-null   object 
 8   hard_drive_type      2201 non-null   object 
 9   hard_drive_capacity  2133 non-null   object 
 10  card                 2203 non-null   object 
 11  card_brand           2203 non-null   object 
 12  screen_size          2183 non-null   object 
 13  screen_type          2095 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 242.4+ KB


In [37]:
ram_counts = train['ram_brand'].value_counts()
print(ram_counts)

ram_brand
DDR5       1143
DDR4        601
LPDDR5      163
LPDDR5X     123
LPDDR4X      36
LPDDR4       12
LPDDR3       11
LDDR4         1
Name: count, dtype: int64


In [38]:
print('trainframe encoded by OHE dimension : ', pd.get_dummies(train, columns = cat_cols, drop_first = True).shape)

trainframe encoded by OHE dimension :  (2215, 683)


In [39]:
# train[cat_cols] = train[cat_cols].fillna('missing')
train.head()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                2212 non-null   object 
 1   price                2141 non-null   float64
 2   old                  2215 non-null   int64  
 3   new                  2215 non-null   int64  
 4   cpu                  2206 non-null   object 
 5   cpu_brand            2213 non-null   object 
 6   ram_capacity         2209 non-null   float64
 7   ram_brand            2090 non-null   object 
 8   hard_drive_type      2201 non-null   object 
 9   hard_drive_capacity  2133 non-null   object 
 10  card                 2203 non-null   object 
 11  card_brand           2203 non-null   object 
 12  screen_size          2183 non-null   object 
 13  screen_type          2095 non-null   object 
dtypes: float64(2), int64(2), object(10)
memory usage: 242.4+ KB


In [40]:
en = LabelEncoder()

for cols in cat_cols:
    train[cols] = en.fit_transform(train[cols])

print('Dataframe encoded by Label encoding dimension : ', train.shape)

Dataframe encoded by Label encoding dimension :  (2215, 14)


In [41]:
train.head()

,brand,price,old,new,cpu,cpu_brand,ram_capacity,ram_brand,hard_drive_type,hard_drive_capacity,card,card_brand,screen_size,screen_type
0,1,26990000.0,0,1,109,2,3,1,5,9,45,14,10,14
1,6,16390000.0,0,1,46,0,3,0,5,9,14,5,17,12
2,1,28990000.0,0,1,109,2,3,1,5,9,45,14,10,14
3,5,36290000.0,0,1,109,2,3,1,5,9,45,14,10,13
4,2,18990000.0,0,1,101,2,1,1,5,9,48,14,10,12


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                2215 non-null   int64  
 1   price                2141 non-null   float64
 2   old                  2215 non-null   int64  
 3   new                  2215 non-null   int64  
 4   cpu                  2215 non-null   int64  
 5   cpu_brand            2215 non-null   int64  
 6   ram_capacity         2215 non-null   int64  
 7   ram_brand            2215 non-null   int64  
 8   hard_drive_type      2215 non-null   int64  
 9   hard_drive_capacity  2215 non-null   int64  
 10  card                 2215 non-null   int64  
 11  card_brand           2215 non-null   int64  
 12  screen_size          2215 non-null   int64  
 13  screen_type          2215 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 242.4 KB


In [43]:
print(train['price'].describe())

count    2.141000e+03
mean     3.028161e+07
std      1.774969e+07
min      6.990000e+06
25%      1.869000e+07
50%      2.499900e+07
75%      3.599000e+07
max      1.489900e+08
Name: price, dtype: float64


In [44]:
print("Max value in y_train:", train.max())
print("Min value in y_train:", train.min())


Max value in y_train: brand                         10.0
price                  148990000.0
old                            1.0
new                            1.0
cpu                          355.0
cpu_brand                      5.0
ram_capacity                  13.0
ram_brand                      8.0
hard_drive_type                6.0
hard_drive_capacity           10.0
card                         192.0
card_brand                    29.0
screen_size                   43.0
screen_type                   20.0
dtype: float64
Min value in y_train: brand                        0.0
price                  6990000.0
old                          0.0
new                          0.0
cpu                          0.0
cpu_brand                    0.0
ram_capacity                 0.0
ram_brand                    0.0
hard_drive_type              0.0
hard_drive_capacity          0.0
card                         0.0
card_brand                   0.0
screen_size                  0.0
screen_type           

In [45]:
def clean_price_data(df):
    # Remove any potential negative prices
    df = df[df['price'] > 0]
    
    # Remove extreme outliers using IQR method
    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    
    df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
    
    return df

In [46]:
train = clean_price_data(train)

In [47]:
X = train.drop('price', axis = 1)
y = train['price']

In [48]:
print(X.shape)
print(y.shape)

(2102, 13)
(2102,)


In [49]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.05,random_state=0)

In [50]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1996, 13)
(1996,)
(106, 13)
(106,)


In [51]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

In [52]:
lgbmr = LGBMRegressor()
xgbr = XGBRegressor(
    n_estimators=100,     # Number of trees
    learning_rate=0.1,    # Step size shrinkage
    max_depth=6,          # Maximum tree depth
    random_state=42,      # Ensures reproducibility
    verbosity=1           # Logs progress
)



In [53]:
if np.any(np.isnan(y_train)):
    print("y_train contains NaN values.")
if np.any(np.isinf(y_train)):
    print("y_train contains infinity values.")


In [54]:
lgbmr.fit(X_train,y_train)
xgbr.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000448 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 438
[LightGBM] [Info] Number of data points in the train set: 1996, number of used features: 12
[LightGBM] [Info] Start training from score 28858593.687375


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [55]:
catboost_model = CatBoostRegressor(cat_features=cat_cols, verbose=0)

In [56]:
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7]
}

# Perform grid search
grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, cv=3, scoring='r2', verbose=1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")


Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'depth': 6, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


In [57]:
catboost_model.fit(X_train, y_train)

In [58]:
best_catboost_model = CatBoostRegressor(cat_features=cat_cols, **best_params, verbose=0)
best_catboost_model.fit(X_train, y_train)

In [59]:
xgbr_pred = xgbr.predict(X_test)
print("r2_score",r2_score(y_test,xgbr_pred))

r2_score 0.831303939529135


In [60]:
lgbmr_pred = lgbmr.predict(X_test)
print("r2_score",r2_score(y_test,lgbmr_pred))

r2_score 0.8515088042591452


In [61]:
y_pred = catboost_model.predict(X_test)
print(f"R2 Score: {catboost_model.score(X_test, y_test)}")

R2 Score: 0.8191376918528955


In [62]:
y_pred = best_catboost_model.predict(X_test)
print(f"R2 Score: {best_catboost_model.score(X_test, y_test)}")

R2 Score: 0.8132331368900079


In [63]:
catboost_model.save_model("model/catboost_model.cbm")

In [64]:
lgbmr.booster_.save_model("model/lightgbm_model.txt")
xgbr.save_model("model/xgboost_model.json")

In [65]:
loaded_cat = CatBoostRegressor()
loaded_cat.load_model("model/catboost_model.cbm")

loaded_xgbr = XGBRegressor()
loaded_xgbr.load_model("model/xgboost_model.json")

import lightgbm as lgb
loaded_lgbmr = lgb.Booster(model_file="model/lightgbm_model.txt")